In [36]:
import pandas as pd
import numpy as pn

df = pd.read_csv("measurements.csv")
df


,gewicht,lange,brustumfang,kreuzbeinhohe,schulterhohe
0,59.4,72.0,88.5,69.0,72.0
1,63.0,75.0,92.0,77.0,76.0
2,68.3,82.0,93.0,75.0,71.5
3,65.8,75.0,90.5,76.0,74.0
4,45.0,76.0,86.0,71.0,68.0
5,49.0,72.0,85.0,69.0,66.0
6,53.0,82.0,68.0,68.0,66.0
7,56.8,74.0,87.0,72.0,65.0
8,45.0,79.5,73.0,73.0,71.0
9,52.2,77.0,85.0,72.0,72.0


In [45]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression

# Create the pipeline
model = Pipeline([
    #('poly_features', PolynomialFeatures(degree=2, include_bias=False)),  # Generate polynomial features
    #('std_scaler', StandardScaler()),  # Standardize the features
    ('lin_reg', LinearRegression())  # Apply linear regression
])
model.fit(df.drop("gewicht", axis=1), df["gewicht"])

Pipeline(steps=[('lin_reg', LinearRegression())])

In [46]:
truth = pd.read_csv("daten.csv")
truth


,gewicht,lange,brustumfang,kreuzbeinhohe,schulterhohe
0,65.9,85,98,73,74
1,82.3,88,105,75,75
2,60.0,85,92,72,72
3,74.5,94,104,77,79
4,34.8,70,79,63,62


In [47]:
from sklearn.metrics import mean_squared_error

result = model.predict(truth.drop("gewicht", axis=1))
print(result)
print(truth["gewicht"])
mse = mean_squared_error(truth["gewicht"], result)
mse

[73.24358913 81.25585589 68.05975423 87.59901817 40.91301883]
0    65.9
1    82.3
2    60.0
3    74.5
4    34.8
Name: gewicht, dtype: float64


65.78629053935929